In [1]:
import requests

In [63]:
benchmarks = ['Public_AR_Current', 'Public_AR_ACS2017', 'Public_AR_Census2010']
vintages = {'Public_AR_Current':
                   ['Current_Current',
                    'Census2010_Current',
                    'ACS2013_Current',
                    'ACS2014_Current',
                    'ACS2015_Current',
                    'ACS2016_Current',
                    'ACS2017_Current',
                    'ACS2018_Current'],
            'Public_AR_ACS2017':
                   ['Current_ACS2018',
                    'Census2010_ACS2018',
                    'ACS2013_ACS2018',
                    'ACS2014_ACS2018',
                    'ACS2015_ACS2018',
                    'ACS2016_ACS2018',
                    'ACS2017_ACS2018',
                    'ACS2018_ACS2018'],
            'Public_AR_Census2010':['Census2010_Census2010','Census2000_Census2010']
           }
returntypes=['locations','geographies']
searchtypes=['onelineaddress','address','coordinates']

class GeocodingParser():
    def __init__(self,benchmark=None,vintage=None):
        self._url = "https://geocoding.geo.census.gov/geocoder/{returntype}/{searchtype}"
        self.benchmark=benchmark if benchmark else benchmarks[0]
        self.vintage=vintage if vintage else vintages[benchmarks[0]][0]
        self.response=None
        
    def get_benchmarks():
        return benchmarks.copy()
    
    def get_vintages(benchmark=None):
        if not benchmark or not benchmark in benchmarks:
            return vintages[benchmarks[0]].copy()
        else:
            return vintages[benchmark].copy()
        
    def search_street(self,address,geo=False,**kwargs):
        '''        https://geocoding.geo.census.gov/geocoder/locations/address?street=4600+Silver+Hill+Rd&
                    city=Suitland&state=MD&zip=20746&benchmark=Public_AR_Census2010&format=json'''
        params={**self._get_params(),**kwargs,'street':address}
        return self._get_response('locations' if not geo else 'geographies','address',params)
    
    def search_onelineaddress(self,geo=False):
        params={**self._get_params(),'address':address}
        return self._get_response('locations' if not geo else 'geographies','onelineaddress',params)
    
    def _get_params(self):
        return {'format':'json','benchmark':self.benchmark,'vintage':self.vintage}
    
    def _get_response(self,returntype,searchtype,params):
        content=None
        with requests.get(self._url.format(returntype=returntype,searchtype=searchtype), params=params) as r:
            content = r.json()
            #print(r.url)
        self.response = content['result']
        return self.response
    
    def get_matches(self):
        return self._get_self_response()['addressMatches']
    
    def get_params(self):
        return self._get_self_response()['input']
    
    def get_first(self):
        return self.get_matches()[0]
    
    def get_first_coordinates(self):
        return self.get_first()['coordinates']
    
    def get_first_geoid(self):
        geographies = self.get_first()['geographies']
        for k in geographies.keys():
            if 'Census' in k:
                break
        return geographies[k][0]['GEOID']
    
    def _get_self_response(self):
        if self.response:
            return self.response
        else:
            raise Exception('No response available.')

In [64]:
gp=GeocodingParser()
gp.search_street('387 Quinby Road',city='Rochester',state='NY',zip=14623,geo=True)
gp.get_first_geoid()

'360550130011008'

In [30]:
import censusgeocode as cg

In [32]:
cg.address('323 Quinby Road',city='Rochester',state='NY',zip=14623)

[{'matchedAddress': '323 QUINBY RD, ROCHESTER, NY, 14623',
  'coordinates': {'x': -77.636536, 'y': 43.108227},
  'tigerLine': {'tigerLineId': '14535048', 'side': 'R'},
  'addressComponents': {'fromAddress': '201',
   'toAddress': '599',
   'preQualifier': '',
   'preDirection': '',
   'preType': '',
   'streetName': 'QUINBY',
   'suffixType': 'RD',
   'suffixDirection': '',
   'suffixQualifier': '',
   'city': 'ROCHESTER',
   'state': 'NY',
   'zip': '14623'},
  'geographies': {'2010 Census Blocks': [{'SUFFIX': '',
     'GEOID': '360550130011008',
     'CENTLAT': '+43.1087162',
     'BLOCK': '1008',
     'AREAWATER': 0,
     'STATE': '36',
     'BASENAME': '1008',
     'OID': 210404043193119,
     'LSADC': 'BK',
     'FUNCSTAT': 'S',
     'INTPTLAT': '+43.1087162',
     'STGEOMETRY.AREA': 2266718.2,
     'STGEOMETRY.LEN': 7373.405,
     'NAME': 'Block 1008',
     'OBJECTID': 5125035,
     'TRACT': '013001',
     'CENTLON': '-077.6340639',
     'BLKGRP': '1',
     'AREALAND': 1207585,
 

In [6]:
GeocodingParser.get_vintages('Public_AR_ACS2017')

['Current_ACS2018',
 'Census2010_ACS2018',
 'ACS2013_ACS2018',
 'ACS2014_ACS2018',
 'ACS2015_ACS2018',
 'ACS2016_ACS2018',
 'ACS2017_ACS2018',
 'ACS2018_ACS2018']

In [7]:
{1:2,3:4}+{7:8}

TypeError: unsupported operand type(s) for +: 'dict' and 'dict'

In [43]:
'Census' in '12312Census 31231'

True